<a href="https://colab.research.google.com/github/codewithselva/industrial-copper-modelling/blob/main/Capstone_Industrial_Copper_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **About the Data:**
1. `id`: This column likely serves as a unique identifier for each transaction or item, which can be useful for tracking and record-keeping.
2. `item_date`: This column represents the date when each transaction or item was recorded or occurred. It's important for tracking the timing of business activities.
3. `quantity tons`: This column indicates the quantity of the item in tons, which is essential for inventory management and understanding the volume of products sold or produced.
4. `customer`: The "customer" column refers to the name or identifier of the customer who either purchased or ordered the items. It's crucial for maintaining customer relationships and tracking sales.
5. `country`: The "country" column specifies the country associated with each customer. This information can be useful for understanding the geographic distribution of customers and may have implications for logistics and international sales.
6. `status`: The "status" column likely describes the current status of the transaction or item. This information can be used to track the progress of orders or transactions, such as "Draft" or "Won."
7. `item type`: This column categorizes the type or category of the items being sold or produced. Understanding item types is essential for inventory categorization and business reporting.
8. `application`: The "application" column defines the specific use or application of the items. This information can help tailor marketing and product development efforts.
9. `thickness`: The "thickness" column provides details about the thickness of the items. It's critical when dealing with materials where thickness is a significant factor, such as metals or construction materials.
10. `width`: The "width" column specifies the width of the items. It's important for understanding the size and dimensions of the products.
11. `material_ref`: This column appears to be a reference or identifier for the material used in the items. It's essential for tracking the source or composition of the products.
12. `product_ref`: The "product_ref" column seems to be a reference or identifier for the specific product. This information is useful for identifying and cataloging products in a standardized way.
13. `delivery date`: This column records the expected or actual delivery date for each item or transaction. It's crucial for managing logistics and ensuring timely delivery to customers.
14. `selling_price`: The "selling_price" column represents the price at which the items are sold. This is a critical factor for revenue generation and profitability analysis.

**Approach: **
1. Data Understanding: Identify the types of variables (continuous, categorical) and their distributions. Some rubbish values are present in ‘Material_Reference’ which starts with ‘00000’ value which should be converted into null. Treat reference columns as categorical variables. INDEX may not be useful.
2. Data Preprocessing:
Handle missing values with mean/median/mode.
Treat Outliers using IQR or Isolation Forest from sklearn library.
Identify Skewness in the dataset and treat skewness with appropriate data transformations, such as log transformation(which is best suited to transform target variable-train, predict and then reverse transform it back to original scale eg:dollars), boxcox transformation, or other techniques, to handle high skewness in continuous variables.
Encode categorical variables using suitable techniques, such as one-hot encoding, label encoding, or ordinal encoding, based on their nature and relationship with the target variable.
3. EDA: Try visualizing outliers and skewness(before and after treating skewness) using Seaborn’s boxplot, distplot, violinplot.
4. Feature Engineering: Engineer new features if applicable, such as aggregating or transforming existing features to create more informative representations of the data. And drop highly correlated columns using SNS HEATMAP.
5. Model Building and Evaluation:
Split the dataset into training and testing/validation sets.
Train and evaluate different classification models, such as ExtraTreesClassifier, XGBClassifier, or Logistic Regression, using appropriate evaluation metrics such as accuracy, precision, recall, F1 score, and AUC curve.
Optimize model hyperparameters using techniques such as cross-validation and grid search to find the best-performing model.
Interpret the model results and assess its performance based on the defined problem statement.
Same steps for Regression modelling.(note: dataset contains more noise and linearity between independent variables so itll perform well only with tree based models)
6. Model GUI: Using streamlit module, create interactive page with
   (1) task input( Regression or Classification) and
   (2) create an input field where you can enter each column value except ‘Selling_Price’ for regression model and  except ‘Status’ for classification model.
   (3) perform the same feature engineering, scaling factors, log/any transformation steps which you used for training ml model and predict this new data from streamlit and display the output.
7. Tips: Use pickle module to dump and load models such as encoder(onehot/ label/ str.cat.codes /etc), scaling models(standard scaler), ML models. First fit and then transform in separate line and use transform only for unseen data
Eg: scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test_new) #unseen data


In [ ]:
!pip install pandas
!pip install numpy
!pip install openpyxl
!pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
excel_file_path = 'copper_data_set.xlsx'

# Specify the sheet name or index
sheet_name_or_index = 'Result 1'  # or use 0 for the first sheet

# Read the Excel file with a specific sheet into a Pandas DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name_or_index)


In [ ]:
# Display the first few rows of the DataFrame

df.head()


In [ ]:
# Total number of records in the data set

len(df)

In [ ]:
# Display the info of the DataFrame

df.info()

**Inference:**
1. Total number of records: 181673
2. item_date field is in float64 Dtype which needs to be converted into date Dtype
3.


In [ ]:
# Create a copy to avoid modifying the original DataFrame
cleaned_df = df.copy()

In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df['item_date'] = pd.to_datetime(cleaned_df['item_date'], format='%Y%m%d', errors='coerce')


In [ ]:

cleaned_df.sort_values(by='item_date', ascending=False, inplace=True)
cleaned_df.tail()

In [ ]:
# Uniform date format

cleaned_df['delivery date'] = pd.to_datetime(cleaned_df['delivery date'], format='%Y%m%d', errors='coerce')

In [ ]:
cleaned_df.sort_values(by='delivery date', ascending=False, inplace=True)
cleaned_df.head()

In [ ]:
# Handling missing values
cleaned_df.dropna(inplace=True)

In [ ]:
cleaned_df.info()

In [ ]:
# Handling duplicate rows
cleaned_df.drop_duplicates(inplace=True)

In [ ]:
cleaned_df.info()

In [ ]:
# Handling outliers (consider replacing 3 with the appropriate threshold)
cleaned_df = cleaned_df[(cleaned_df['quantity tons'].between(cleaned_df['quantity tons'].quantile(0.01), cleaned_df['quantity tons'].quantile(0.99))) &
                        (cleaned_df['thickness'].between(cleaned_df['thickness'].quantile(0.01), cleaned_df['thickness'].quantile(0.99)))]

In [ ]:
# Checking for consistency in categorization
# Assuming 'status', 'item_type', 'application' are categorical columns
cleaned_df['status'] = cleaned_df['status'].astype('category')
cleaned_df['item type'] = cleaned_df['item type'].astype('category')
cleaned_df['application'] = cleaned_df['application'].astype('category')

In [ ]:
# Some rubbish values are present in ‘Material_Reference’ which starts with ‘00000’ value which should be converted into null
cleaned_df['material_ref'] = cleaned_df['material_ref'].apply(lambda x: None if str(x).startswith('0000') else x)


In [ ]:
# material_ref - is a category
cleaned_df['material_ref'] = cleaned_df['material_ref'].astype('category')

In [ ]:
# Display basic statistics

cleaned_df.describe()

In [18]:
# Visualize skewness
sns.displot(cleaned_df['quantity tons'], kde=True)
plt.title('Distribution of Quantity Tons')
plt.show()

In [ ]:
# Visualize outliers using boxplot
sns.boxplot(x=cleaned_df['quantity tons'])
plt.title('Boxplot of Quantity Tons')
plt.show()

In [ ]:

# Identify the row causing the issue
problematic_row = cleaned_df.iloc[173086]

# Print the values in the problematic row
print(problematic_row)